In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import logging
import numpy as np
import socket
hostname = socket.gethostname()

import sys
import mxnet as mx

In [ ]:
labels = ['5N', '7n', '7N', '12N', 'Gr', 'LVe', 'Pn', 'SuVe', 'VLL' , 'BackG']
label_dict = dict([(l,i) for i, l in enumerate(labels)])

In [ ]:
logging.basicConfig(level=logging.DEBUG)

stack = 'MD589'
sec = 195

if hostname.startswith('gcn'):
    data_dir = '/oasis/projects/nsf/csd395/yuncong/CSHL_data_patches/%s_byLandmark'%(stack)
    model_dir = '/oasis/projects/nsf/csd395/yuncong/jiaxu_repo/model_publish'
elif hostname == 'yuncong-MacbookPro':
    data_dir = '/home/yuncong/CSHL_data_patches/%s_byLandmark'%(stack)
    model_dir = '/home/yuncong/inception_model' 
    init_model_dir = '/home/yuncong/jiaxu_repo/model_publish'

In [ ]:
dataset = '%s'%(stack)

In [ ]:
import random

In [ ]:
all_image_names = []
for label in os.listdir(data_dir):
    if label == 'BackG':
        # limit the number of BackG examples
        for img_filename in os.listdir(data_dir + '/' + label)[:300]:
            all_image_names.append((label_dict[label], label + '/' + img_filename))
    else:            
        for img_filename in os.listdir(data_dir + '/' + label):
            all_image_names.append((label_dict[label], label + '/' + img_filename))
        
random.shuffle(all_image_names)
n = len(all_image_names)

In [ ]:
with open('/home/yuncong/CSHL_data_patches/%s_train.lst'%(dataset), 'w') as lst_file:
    for index, (l, fn) in enumerate(all_image_names[:n/2]):
        lst_file.write('%s\t%d\t%s\n'%(index, l, fn))
        
with open('/home/yuncong/CSHL_data_patches/%s_eval.lst'%(dataset), 'w') as lst_file:
    for index, (l, fn)  in enumerate(all_image_names[n/2:n*3/4]):
        lst_file.write('%s\t%d\t%s\n'%(index, l, fn))
        
with open('/home/yuncong/CSHL_data_patches/%s_test.lst'%(dataset), 'w') as lst_file:
    for index, (l, fn)  in enumerate(all_image_names[n/2:n*3/4]):
        lst_file.write('%s\t%d\t%s\n'%(index, l, fn))

In [ ]:
os.system(os.environ['MXNET_DIR']+'/bin/im2rec /home/yuncong/CSHL_data_patches/%(dataset)s_train.lst \
%(data_dir)s/ \
/home/yuncong/CSHL_data_patches/%(dataset)s_train.rec' % \
          {'dataset': dataset,
          'data_dir': data_dir})

In [ ]:
os.system(os.environ['MXNET_DIR']+'/bin/im2rec /home/yuncong/CSHL_data_patches/%(dataset)s_eval.lst \
%(data_dir)s/ \
/home/yuncong/CSHL_data_patches/%(dataset)s_eval.rec' % \
          {'dataset': dataset,
          'data_dir': data_dir})

# on mbp, scanning 1000 images take 1.8s
# on gordon, scanning 16 image, 0.06s

In [ ]:
os.system(os.environ['MXNET_DIR']+'/bin/im2rec /home/yuncong/CSHL_data_patches/%(dataset)s_test.lst \
%(data_dir)s/ \
/home/yuncong/CSHL_data_patches/%(dataset)s_test.rec' % \
          {'dataset': dataset,
          'data_dir': data_dir})

In [ ]:
batch_size = 16

In [ ]:
train_iter = mx.io.ImageRecordIter(
    path_imgrec='/home/yuncong/CSHL_data_patches/%(dataset)s_train.rec' % {'dataset': dataset},
    batch_size=batch_size,
    data_shape=(3, 224, 224),
    mean_img=os.path.join(model_dir, 'mean_224.nd'),
)

eval_iter = mx.io.ImageRecordIter(
    path_imgrec='/home/yuncong/CSHL_data_patches/%(dataset)s_eval.rec' % {'dataset': dataset},
    batch_size=batch_size,
    data_shape=(3, 224, 224),
    mean_img=os.path.join(model_dir, 'mean_224.nd'),
)

test_iter = mx.io.ImageRecordIter(
    path_imgrec='/home/yuncong/CSHL_data_patches/%(dataset)s_test.rec' % {'dataset': dataset},
    batch_size=batch_size,
    data_shape=(3, 224, 224),
    mean_img=os.path.join(model_dir, 'mean_224.nd'),
)

In [ ]:
n_iter = 6
n_class = 10
prefix = 'inception-stage1'

init_model = mx.model.FeedForward.load(os.path.join(init_model_dir, prefix), n_iter, ctx=mx.gpu())
internals = init_model.symbol.get_internals()
symbol = internals['flatten_output']
symbol = mx.symbol.FullyConnected(data=symbol, name='fullc', num_hidden=n_class)
symbol = mx.symbol.SoftmaxOutput(data=symbol, name='softmax')

In [ ]:
n_epoch = 10
l_rate = 0.001
checkpoint_prefix = os.path.join(model_dir, 'inception-stage1')

In [ ]:
opt = mx.optimizer.SGD(learning_rate=l_rate)
net = mx.model.FeedForward(ctx=mx.gpu(), symbol=symbol, num_epoch=n_epoch, optimizer=opt,
                                arg_params=init_model.arg_params, aux_params=init_model.aux_params,
                                allow_extra_params=True)

In [ ]:
net.fit(train_iter, eval_data=eval_iter,
        batch_end_callback=mx.callback.Speedometer(batch_size, 30),
        epoch_end_callback=mx.callback.do_checkpoint(checkpoint_prefix))

net.save(checkpoint_prefix)